In [1]:
conda install -c conda-forge ffmpeg libsndfile

Solving environment: done

## Package Plan ##

  environment location: /Users/cooky/miniforge3

  added / updated specs:
    - ffmpeg
    - libsndfile


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2022.6.15  |       h4653dfc_0         149 KB  conda-forge
    certifi-2022.6.15          |   py39h2804cbe_0         155 KB  conda-forge
    conda-4.13.0               |   py39h2804cbe_1         992 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.3 MB

The following packages will be UPDATED:

  ca-certificates                    2022.5.18.1-h4653dfc_0 --> 2022.6.15-h4653dfc_0
  certifi                        2022.5.18.1-py39h2804cbe_0 --> 2022.6.15-py39h2804cbe_0
  conda                               4.12.0-py39h2804cbe_0 --> 4.13.0-py39h2804cbe_1



conda-4.13.0         | 992 KB    |

In [6]:
!apt install ffmpeg

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.



In [3]:
import tensorflow

TypeError: Unable to convert function return value to a Python type! The signature was
	() -> handle

In [ ]:
tf

In [3]:
from IPython.display import Audio

In [4]:
!wget https://github.com/deezer/spleeter/raw/master/audio_example.mp3

--2022-06-20 17:38:05--  https://github.com/deezer/spleeter/raw/master/audio_example.mp3
github.com (github.com) 해석 중... 52.69.186.44
다음으로 연결 중: github.com (github.com)|52.69.186.44|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/deezer/spleeter/master/audio_example.mp3 [따라감]
--2022-06-20 17:38:05--  https://raw.githubusercontent.com/deezer/spleeter/master/audio_example.mp3
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 262867 (257K) [audio/mpeg]
저장 위치: `audio_example.mp3'

audio_example.mp3   100%[===================>] 256.71K   825KB/s    /  0.3s    

2022-06-20 17:38:07 (825 KB/s) - `audio_example.mp3' 저장함 [262867/262867]



In [4]:

spleeter

NameError: name 'spleeter' is not defined

In [2]:
import tensorflow as tf

In [3]:
tf.__version__

'2.5.0'

In [4]:
!wget https://github.com/deezer/spleeter/raw/master/audio_example.mp3

--2022-06-20 23:26:45--  https://github.com/deezer/spleeter/raw/master/audio_example.mp3
github.com (github.com) 해석 중... 15.164.81.167
다음으로 연결 중: github.com (github.com)|15.164.81.167|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 302 Found
위치: https://raw.githubusercontent.com/deezer/spleeter/master/audio_example.mp3 [따라감]
--2022-06-20 23:26:45--  https://raw.githubusercontent.com/deezer/spleeter/master/audio_example.mp3
raw.githubusercontent.com (raw.githubusercontent.com) 해석 중... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
다음으로 연결 중: raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... 연결했습니다.
HTTP 요청을 보냈습니다. 응답 기다리는 중... 200 OK
길이: 262867 (257K) [audio/mpeg]
저장 위치: `audio_example.mp3.1'

audio_example.mp3.1 100%[===================>] 256.71K  --.-KB/s    /  0.05s   

2022-06-20 23:26:46 (5.50 MB/s) - `audio_example.mp3.1' 저장함 [262867/262867]



In [6]:
!spleeter separate -o output/ audio_example.mp3

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB

INFO:spleeter:File output/audio_example/accompaniment.wav written succesfully
INFO:spleeter:File output/audio_example/vocals.wav written succesfully
